In [19]:
##Setup
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf
from IPython.display import Audio


device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler_tts_mini_v0.1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler_tts_mini_v0.1")


c:\Users\setht\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [23]:
import requests
import json

# Define your prompt
prompt = "you are an AI assistant designed to help, with short consise responses. "+"Hello, how are you?"

# Prepare JSON payload
data = {
    "model": "starling-lm", 
    # "prompt": prompt, 
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant who is consise with your responses"
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    "stream": False
}
response = requests.post('http://localhost:11434/api/chat', json=data)
# print(response.content)
result = response.json()
starling_output = result["message"]["content"]
print(starling_output)


Hello! I'm doing well, thank you for asking. As an AI language model, my primary purpose is to assist and provide helpful information in a concise manner. If you have any questions or need assistance, feel free to ask!


In [22]:

response_text = starling_output
description = "A male speaker delivers his words quite expressively, in a very confined sounding environment with clear audio quality. he speaks very confidintly."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(response_text, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)

Audio("parler_tts_out.wav")



Using the model-agnostic default `max_length` (=2580) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
